In [1]:
# Puts last week's data into Google Sheets

# from here: http://www.compciv.org/guides/python/how-tos/creating-proper-url-query-strings/

import requests

import pandas as pd
from pandas import json_normalize

import json

from dotenv import load_dotenv
load_dotenv()

import datetime
from datetime import timedelta
from datetime import date

import gspread
from gspread_dataframe import set_with_dataframe
import gspread_dataframe as gd

from df2gspread import df2gspread as d2g

import os

from gspread_pandas import spread, client

In [2]:
# environment variable(s). Used this Youtube video: https://www.youtube.com/watch?v=YdgIWTYQ69A

from textwrap import indent

harvest_entries = []
page = 1
per_page = 100

while True:
    # print("----")
    # print("Requesting", url)

    payload={}

    url = f"https://api.harvestapp.com/api/v2/time_entries.json?page={page}&per_page={per_page}"

    headers = {
        "User-Agent": "Python Harvest API Sample",
        "Authorization": "Bearer " + os.environ.get("HARVEST_ACCESS_TOKEN"),
        "Harvest-Account-ID": os.environ.get("HARVEST_ACCOUNT_ID"),
        "from": "begin_last_wk",
        "to": "end_last_wk",
    }

    response = requests.get(url, headers=headers, data=payload)
    data = response.json()

    time_entries = data.get('time_entries')

    print(len(time_entries))

    # Did we find any new results, or did we run out?
    if len(time_entries) == 0:
        # If not, exit the loop
        break

    harvest_entries.extend(time_entries)

    page = page + 1



100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
63
0


In [3]:
# Flattens nested JSON data]
df = json_normalize(harvest_entries)

df.columns.tolist()

['id',
 'spent_date',
 'hours',
 'hours_without_timer',
 'rounded_hours',
 'notes',
 'is_locked',
 'locked_reason',
 'is_closed',
 'is_billed',
 'timer_started_at',
 'started_time',
 'ended_time',
 'is_running',
 'billable',
 'budgeted',
 'billable_rate',
 'cost_rate',
 'created_at',
 'updated_at',
 'invoice',
 'external_reference',
 'user.id',
 'user.name',
 'client.id',
 'client.name',
 'client.currency',
 'project.id',
 'project.name',
 'project.code',
 'task.id',
 'task.name',
 'user_assignment.id',
 'user_assignment.is_project_manager',
 'user_assignment.is_active',
 'user_assignment.use_default_rates',
 'user_assignment.budget',
 'user_assignment.created_at',
 'user_assignment.updated_at',
 'user_assignment.hourly_rate',
 'task_assignment.id',
 'task_assignment.billable',
 'task_assignment.is_active',
 'task_assignment.created_at',
 'task_assignment.updated_at',
 'task_assignment.hourly_rate',
 'task_assignment.budget',
 'invoice.id',
 'invoice.number',
 'external_reference.id',


In [4]:

# Build dataframe from only the columns I want.

df = df[['spent_date','project.name','project.code','cost_rate','billable_rate','hours']]

df

,spent_date,project.name,project.code,cost_rate,billable_rate,hours
0,2022-09-30,RSD-Time Off Resound [G] 18RSD15H,18RSD15H,70.0,NaN,8.00
1,2022-09-26,RSD-Podcast Resound [G] 17RSD09H,17RSD09H,32.0,70.0,0.25
2,2022-09-26,RSD-Internal Project Resound Accounting Firm R...,22RSD17F,15.0,NaN,1.00
3,2022-09-26,UDA-OneDairy Brand Dev United Dairymen of Ariz...,22UDA22F,32.0,175.0,0.08
4,2022-09-26,SUN-Rebrand + Website Package Sunhome [J] 22SU...,22SUN27F,32.0,175.0,0.08
...,...,...,...,...,...,...
16058,2018-07-10,MAC-Monthly Ongoing [J] 17MAC04F,17MAC04F,NaN,150.0,0.75
16059,2018-07-10,INF-PCON Website Design and Dev Infusionsoft [...,18INF06F,NaN,150.0,1.00
16060,2018-07-10,MKT-Instructional Design Mark Taylor [J] 18MKT02H,18MKT02F,NaN,150.0,2.00
16061,2018-07-09,INF-PCON Website Design and Dev Infusionsoft [...,18INF06F,NaN,150.0,0.75


In [5]:
# Remove invalid JSON
df.fillna('0',inplace=True)

df

,spent_date,project.name,project.code,cost_rate,billable_rate,hours
0,2022-09-30,RSD-Time Off Resound [G] 18RSD15H,18RSD15H,70.0,0,8.00
1,2022-09-26,RSD-Podcast Resound [G] 17RSD09H,17RSD09H,32.0,70.0,0.25
2,2022-09-26,RSD-Internal Project Resound Accounting Firm R...,22RSD17F,15.0,0,1.00
3,2022-09-26,UDA-OneDairy Brand Dev United Dairymen of Ariz...,22UDA22F,32.0,175.0,0.08
4,2022-09-26,SUN-Rebrand + Website Package Sunhome [J] 22SU...,22SUN27F,32.0,175.0,0.08
...,...,...,...,...,...,...
16058,2018-07-10,MAC-Monthly Ongoing [J] 17MAC04F,17MAC04F,0,150.0,0.75
16059,2018-07-10,INF-PCON Website Design and Dev Infusionsoft [...,18INF06F,0,150.0,1.00
16060,2018-07-10,MKT-Instructional Design Mark Taylor [J] 18MKT02H,18MKT02F,0,150.0,2.00
16061,2018-07-09,INF-PCON Website Design and Dev Infusionsoft [...,18INF06F,0,150.0,0.75


Push DF to Google Sheets

In [6]:
# define worksheet from Google Sheets

sa = gspread.service_account(filename="/Users/chrisstadler/Dev for Resound/TextEdit/service_account.json")
sh = sa.open("Harvest Time Entries JSON")

wks_totals = sh.worksheet("Downloaded Time Entries")

In [7]:
# The actual update of the spreadsheet

# Clears columns
wks_totals.batch_clear(['A:K'])

# Turns pandas dataframe into list that can be uploaded via gspread.Cell
# Also adds headers, so it's all in one list and ready to upload
headers = df.columns.values.tolist()
body = df.values.tolist()
body.insert(0,headers)

# print(body[0:3])


In [8]:
# wks_totals.update(body)

wks_totals.update(body)

{'spreadsheetId': '1S5uZ4tA4jq6LjKcnNAXcMfhnVNpYD6BeYPxQPKQ5Edc',
 'updatedRange': "'Downloaded Time Entries'!A1:F16064",
 'updatedRows': 16064,
 'updatedColumns': 6,
 'updatedCells': 96384}